In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/datos para el tp2/set_datos_propiedades.csv')

In [3]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull())\
                              & (propiedades.lat.notnull()) & (propiedades.lon.notnull()),\
                             ['lat', 'lon', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [4]:
columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [5]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [122]:
cantidad_clusters = 40

cluster = KMeans(n_clusters = cantidad_clusters)
m = cluster.fit_predict(set_entrenamiento_datos,set_entrenamiento_resultado)

In [123]:
set_entrenamiento.loc[:, 'cluster'] = m

# Con decision Tree

In [62]:
from sklearn.tree import DecisionTreeRegressor

In [124]:
criterio = 'mse'
max_depth = None
min_sample = 2
min_leaf = 1
feature = 1.0

predictores = {}
for i in range (cantidad_clusters):
    tree = DecisionTreeRegressor(criterion = criterio, max_depth = max_depth,\
                                           min_samples_split = min_sample, min_samples_leaf = min_leaf,\
                                           max_features = feature)
    entrenamiento = set_entrenamiento.loc[set_entrenamiento.cluster == i,:]
    datos = entrenamiento.loc[:, columnas]
    resultado = entrenamiento.loc[:,'price_aprox_usd']
    tree.fit(datos, resultado)
    predictores[i] = tree

In [120]:
def predecir(x, predictor, clusters):
    indice = cluster.predict([x])[0]
    return predictor[indice].predict([x])[0]

In [125]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x,predictores,cluster),axis = 1)

error = mean_squared_error(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("error = {}, cant_digitos = {}".format(error, len(str(int(round(2306842941.06,0))))))

error = 2650306881.35, cant_digitos = 10
